In [1]:
print("hello world")

hello world


In [4]:
!pip install opencv-python


  Using cached opencv_python-4.7.0.72-cp37-abi3-win_amd64.whl (38.2 MB)


In [6]:
!pip install albumentations

     -------------------------------------- 123.5/123.5 kB 3.5 MB/s eta 0:00:00
     -------------------------------------- 42.5/42.5 MB 899.2 kB/s eta 0:00:00
     ---------------------------------------- 23.7/23.7 MB 1.9 MB/s eta 0:00:00
     -------------------------------------- 151.7/151.7 kB 8.8 MB/s eta 0:00:00
     ---------------------                    20.2/38.1 MB 2.3 MB/s eta 0:00:08


ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device



In [5]:
import cv2
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
from decimal import Decimal
import albumentations as A

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Convert box coordinate from yolo to voc
def yoloFormattocv(x1, y1, x2, y2, H, W):
    bbox_width = x2 * W
    bbox_height = y2 * H
    center_x = x1 * W
    center_y = y1 * H
    voc = []
    voc.append(center_x - (bbox_width / 2))
    voc.append(center_y - (bbox_height / 2))
    voc.append(center_x + (bbox_width / 2))
    voc.append(center_y + (bbox_height / 2))
    return [int(v) for v in voc]

In [ ]:
# Plot yolo box
box_color = {0:'blue',1:'orange',2:'green',3:'purple',4:'brown',5:'pink',
             6:'gray',7:'olive',8:'cyan',9:'gold',10:'turquoise',11:'navy',12:'yellow',13:'red'}
box_name = {0:'RazorBlade',1:'SafetyPin',2:'PaperClip',3:'Pen',4:'ThinNail',5:'Screw',
            6:'HandGun',7:'VgaConnector',8:'Knife',9:'MultiPurposeKnife',10:'Key',11:'Plier',12:'Shuriken',13:'Scissor'}
def plot_box(box_yolo_coordinates, height, width):
  ''' Input: list of boxes
      Output: plot of boxes'''
  for row in box_yolo_coordinates:
    bbox = row.strip('\n').split(' ')
    [x1,y1,x2,y2] = yoloFormattocv(Decimal(bbox[1]),Decimal(bbox[2]),Decimal(bbox[3]),Decimal(bbox[4]),height, width)
    plt.gca().add_patch(Rectangle((x1,y1),x2-x1,y2-y1, edgecolor=box_color[int(bbox[0])], facecolor='none', lw=1))
    plt.annotate(box_name[int(bbox[0])], ((x2+x1)/2, (y2+y1)/2), 
                 color=box_color[int(bbox[0])], weight='bold', fontsize=10, ha='center', va='center') # label at the center of the box

In [ ]:
def cvFormattoYolo(corner, H, W):
    ''' Input: list of coordinates in voc format ['label','x1','y1','x2','y2']
        Output: list of coordinates in yolo format 'label x_center y_center box_width box_height' '''
    bbox_W = corner[3] - corner[1]
    bbox_H = corner[4] - corner[2]

    center_bbox_x = (corner[1] + corner[3]) / 2
    center_bbox_y = (corner[2] + corner[4]) / 2

    return corner[0]+' '+str(round(center_bbox_x / W, 6))+' '+str(round(center_bbox_y / H, 6))+' '+str(round(bbox_W / W, 6))+' '+str(round(bbox_H / H, 6))

In [ ]:
# Plot test image and its box
image_name = 'B0072_0001'
image_ext = '.png'
image = cv2.imread(image_name + image_ext)
plt.rcParams["figure.figsize"] = (20,10)
plt.imshow(image)

height, width,_ = image.shape
f = open(image_name + '.txt', 'r')
box_list = f.readlines()
plot_box(box_list, height, width)
box_list

In [ ]:
# Rotate 90 degree
im = yoloRotatebbox(image_name, image_ext, 90)
image_30 = im.rotate_image()
bbox_30 = im.rotateYolobbox()

plt.rcParams["figure.figsize"] = (20,10)
plt.imshow(image_30)
plot_box(bbox_30, height, width)

In [ ]:
# Rotate 30 degree
im = yoloRotatebbox(image_name, image_ext, 30)
image_30 = im.rotate_image()
bbox_30 = im.rotateYolobbox()

plt.rcParams["figure.figsize"] = (30,15)
plt.imshow(image_30)
plot_box(bbox_30, height, width)

In [ ]:
transform = A.Compose([

    A.RandomCrop(width=450, height=450),
    A.ShiftScaleRotate(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.MedianBlur(blur_limit=7, always_apply=False, p=0.2),

    
],bbox_params=A.BboxParams(format='yolo'))